# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/Part01/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [3]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [19]:
def get_cross_val_score(df, train_num, train_Y, _cv=5):
    train_tempX = df[:train_num]
    estimator_temp = LogisticRegression(solver='lbfgs')
    cvs_temp = cross_val_score(estimator_temp, train_tempX, train_Y, cv=_cv).mean()
    
    return cvs_temp
    

# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [27]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
#print(df_m1.head(10),"\n\n")
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cvs1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()


"""
Your Code Here
"""
# 空值補 平均數，做邏輯斯迴歸
df_m2 = df.fillna(df.mean())
cvs2 = get_cross_val_score(df_m2
                           ,train_num, train_Y)

# 空值補 0，做邏輯斯迴歸
df_m3 = df.fillna(0)
cvs3 = get_cross_val_score(df_m3
                           ,train_num, train_Y)

# 空值補 標準差，做邏輯斯迴歸
df_m4 = df.fillna(df.std())
cvs4 = get_cross_val_score(df_m4
                           ,train_num, train_Y)

# 空值補 1，做邏輯斯迴歸
df_m5 = df.fillna(1)
cvs5 = get_cross_val_score(df_m5 
                           ,train_num, train_Y)

# 空值補 中位數，做邏輯斯迴歸
df_m6 = df.fillna(df.median())
cvs6 = get_cross_val_score(df_m6
                           ,train_num, train_Y)


print('[原值]','='*60)
print(f'cross_val_score 1: {cvs1}')
print(f'cross_val_score 2: {cvs2}')
print(f'cross_val_score 3: {cvs3}')
print(f'cross_val_score 4: {cvs4}')
print(f'cross_val_score 5: {cvs5}')
print(f'cross_val_score 6: {cvs6}')

[原值] ============================================================
cross_val_score 1: 0.6982644788418415
cross_val_score 2: 0.6959413955734954
cross_val_score 3: 0.6993817972775958
cross_val_score 4: 0.6926084842578535
cross_val_score 5: 0.6993817972775958
cross_val_score 6: 0.6970587140092496


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

> ANS：<font color="blue">目前比較起來，再經過 最小最大化`MinMaxScaler()` 後的結果比較好。</font>

In [30]:
"""
Your Code Here
"""

# 將數值再進行 最小最大化
cvs1mms = get_cross_val_score(MinMaxScaler().fit_transform(df_m1.astype('float'))
                           ,train_num, train_Y)

cvs2mms = get_cross_val_score(MinMaxScaler().fit_transform(df_m2.astype('float'))
                           ,train_num, train_Y)

cvs3mms = get_cross_val_score(MinMaxScaler().fit_transform(df_m3.astype('float'))
                           ,train_num, train_Y)

cvs4mms = get_cross_val_score(MinMaxScaler().fit_transform(df_m4.astype('float'))
                           ,train_num, train_Y)

cvs5mms = get_cross_val_score(MinMaxScaler().fit_transform(df_m5.astype('float'))
                           ,train_num, train_Y)

cvs6mms = get_cross_val_score(MinMaxScaler().fit_transform(df_m6.astype('float'))
                           ,train_num, train_Y)


print('[最大最小化]','='*60)
print(f'cross_val_score with MinMaxScaler 1: {cvs1mms}')
print(f'cross_val_score with MinMaxScaler 2: {cvs2mms}')
print(f'cross_val_score with MinMaxScaler 3: {cvs3mms}')
print(f'cross_val_score with MinMaxScaler 4: {cvs4mms}')
print(f'cross_val_score with MinMaxScaler 5: {cvs5mms}')
print(f'cross_val_score with MinMaxScaler 6: {cvs6mms}')


[最大最小化] ============================================================
cross_val_score with MinMaxScaler 1: 0.7005053927832138
cross_val_score with MinMaxScaler 2: 0.6993439220763837
cross_val_score with MinMaxScaler 3: 0.7038636251603401
cross_val_score with MinMaxScaler 4: 0.7015974610848266
cross_val_score with MinMaxScaler 5: 0.7049872206659581
cross_val_score with MinMaxScaler 6: 0.700461240512138


In [29]:
# 將數值再進行 標準化

cvs1std = get_cross_val_score(StandardScaler().fit_transform(df_m1.astype('float'))
                           ,train_num, train_Y)

cvs2std = get_cross_val_score(StandardScaler().fit_transform(df_m2.astype('float'))
                           ,train_num, train_Y)

cvs3std = get_cross_val_score(StandardScaler().fit_transform(df_m3.astype('float'))
                           ,train_num, train_Y)

cvs4std = get_cross_val_score(StandardScaler().fit_transform(df_m4.astype('float'))
                           ,train_num, train_Y)

cvs5std = get_cross_val_score(StandardScaler().fit_transform(df_m5.astype('float'))
                           ,train_num, train_Y)

cvs6std = get_cross_val_score(StandardScaler().fit_transform(df_m6.astype('float'))
                           ,train_num, train_Y)


print('[標準化]','='*60)
print(f'cross_val_score with StandardScaler 1: {cvs1std}')
print(f'cross_val_score with StandardScaler 2: {cvs2std}')
print(f'cross_val_score with StandardScaler 3: {cvs3std}')
print(f'cross_val_score with StandardScaler 4: {cvs4std}')
print(f'cross_val_score with StandardScaler 5: {cvs5std}')
print(f'cross_val_score with StandardScaler 6: {cvs6std}')


[標準化] ============================================================
cross_val_score with StandardScaler 1: 0.6982582017719778
cross_val_score with StandardScaler 2: 0.6959413955734954
cross_val_score with StandardScaler 3: 0.6982582017719778
cross_val_score with StandardScaler 4: 0.6937320797634714
cross_val_score with StandardScaler 5: 0.6971346062663598
cross_val_score with StandardScaler 6: 0.6970587140092496
